# Digital-Contact-Tracing on DTU

This notebook demonstrates the use of the simulator on the DTU dataset.

### Imports and setup

In [1]:
import numpy as np
import DigitalContactTracing as DCT
from load_temporal_graph import load_df, build_graphs, get_array_of_contacts

%load_ext autoreload
%autoreload 2

### Define the parameters of the simulation

In [2]:
PARAMETERS = {"temporal_gap" : 900,                 # Temporal gap between static networks (sec)
              "memory_contacts" : 7,                # Tracing memory (days)
              "max_time_quar" : 10,                 # Quarantine duration (days)
              "policies" : {"policy1":[-100,600]},  # Digital tracing policies [RSSI, duration (sec)]
              "eps_Is" : [0.5,0.2],                 # Isolation effectivity
              "times" : 2,                          # Number of repetition of the simulation
              "seed" : 11,                          # Random seed 
              "Y_i" : 10,                           # Number of initially infected people
              "nc" : 0.2,                           # Fraction of individuals who are not adopting the app
              "beta_t": 0.002,                      # Parameter defining the infectiousness probability
              "symptomatics" : 0.8,                 # Fraction of symptomatic individuals 
              "testing" : 0.25,                     # Fraction of asymptomatics who are detected via random testing
              "store":{"to_store": True,            # Save the results?
                       "path_to_store":"RESULTS/"}} # Target folder to save the results

### Load the temporal graph

In [16]:
df = load_df("Dataset//bt_symmetric.csv", n_row=2000000)
graphs = build_graphs(get_array_of_contacts(df, PARAMETERS["temporal_gap"], column_name='# timestamp'), 
                      PARAMETERS["temporal_gap"])

(2426279, 4)


### Run the simulations and store the results

In [17]:
# Initialize the random seed
np.random.seed(PARAMETERS["seed"])

# Run over each configuration
for policy in PARAMETERS["policies"]:
    for eps_I in PARAMETERS["eps_Is"]:
        filter_rssi = PARAMETERS["policies"][policy][0]
        filter_duration = PARAMETERS["policies"][policy][1]
        results = []
        
        # Print some report
        print("-" * 100)
        print('Running simulation with policy: %s' % policy)
        print('       |_ eps_I             : %+4.2f'   % eps_I)
        print('       |_ filter_rssi       : %+4.2f' % filter_rssi)
        print('       |_ filter_duration   : %+4.2f' % filter_duration)
        print('')
        
        # Repeat the simulation for a fixed configuration 
        for i in range(PARAMETERS["times"]):
            # Print some report
            print('       Running simulation     [%4d / %4d]' % (i + 1, PARAMETERS['times']))

            # Initialize the simulation object
            dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration)
            # Run the actual simulation
            res = dct.simulate()
            
            # Save the results    
            if PARAMETERS["store"]["to_store"]:    
                DCT.store_real_time(res,PARAMETERS,filter_rssi,filter_duration,eps_I)

----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I             : +0.50
       |_ filter_rssi       : -100.00
       |_ filter_duration   : +600.00

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]
----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I             : +0.20
       |_ filter_rssi       : -100.00
       |_ filter_duration   : +600.00

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]


In [6]:
import cProfile
dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration)
cProfile.run('res = dct.simulate()', 'execution_stats')
import pstats
from pstats import SortKey
p = pstats.Stats('execution_stats')
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(30)

Thu Jul  9 07:53:45 2020    execution_stats

         16171067 function calls in 7.507 seconds

   Ordered by: cumulative time
   List reduced from 56 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.507    7.507 {built-in method builtins.exec}
        1    0.000    0.000    7.507    7.507 <string>:1(<module>)
        1    0.091    0.091    7.507    7.507 DigitalContactTracing.py:233(simulate)
     1116    0.630    0.001    6.424    0.006 DigitalContactTracing.py:523(update_contacts)
   751068    1.769    0.000    5.399    0.000 DigitalContactTracing.py:371(policy)
  1848078    0.922    0.000    2.888    0.000 graph.py:451(__getitem__)
  1848078    0.874    0.000    1.087    0.000 graph.py:349(adj)
  1848078    0.691    0.000    0.879    0.000 coreviews.py:80(__getitem__)
     1116    0.079    0.000    0.618    0.001 DigitalContactTracing.py:311(update_infected)
    23134    0.092    0.000    0.458  

In [23]:
import networkx as nx
A_duration = []
A_rssi = []
A_policy = []
for graph in graphs:
    A_duration.append(nx.adjacency_matrix(graph, weight='duration'))
    A_rssi.append(nx.adjacency_matrix(graph, weight='rssi'))
    A_policy.append()

In [25]:
A[0] < 2

/home/gab/anaconda3/lib/python3.7/site-packages/scipy/sparse/compressed.py:287: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


<108x108 sparse matrix of type '<class 'numpy.bool_'>'
	with 11664 stored elements in Compressed Sparse Row format>